In [ ]:
!pip install ptflops
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import torchvision.models as models
import time
import pandas as pd
import numpy as np
from torch.utils.data import Subset

from ptflops import get_model_complexity_info

In [ ]:
device_cpu = torch.device("cpu")
device_gpu = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print("GPU Available:", torch.cuda.is_available())

GPU Available: True


In [ ]:
batch_size = 16

transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=3),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=(0.5, 0.5, 0.5),
        std=(0.5, 0.5, 0.5)
    )
])

train_dataset = torchvision.datasets.FashionMNIST(
    root="./data", train=True, download=True, transform=transform
)

test_dataset = torchvision.datasets.FashionMNIST(
    root="./data", train=False, download=True, transform=transform
)

train_size = int(0.3 * len(train_dataset))
train_indices = np.random.choice(len(train_dataset), train_size, replace=False)
train_dataset = Subset(train_dataset, train_indices)

test_size = int(0.3 * len(test_dataset))
test_indices = np.random.choice(len(test_dataset), test_size, replace=False)
test_dataset = Subset(test_dataset, test_indices)


train_loader = torch.utils.data.DataLoader(
    train_dataset, batch_size=batch_size, shuffle=True
)

test_loader = torch.utils.data.DataLoader(
    test_dataset, batch_size=batch_size, shuffle=False
)

In [ ]:
def get_model(model_name):
    if model_name == "resnet18":
        model = models.resnet18(pretrained=False)
    elif model_name == "resnet32":
        model = models.resnet34(pretrained=False)
    elif model_name == "resnet50":
        model = models.resnet50(pretrained=False)
    else:
        raise ValueError("Invalid model")

    model.fc = nn.Linear(model.fc.in_features, 10)
    return model

In [ ]:
def compute_flops(model):
    model.eval()
    flops, params = get_model_complexity_info(
        model,
        (3, 224, 224),
        as_strings=False,
        print_per_layer_stat=False
    )
    return flops

In [ ]:
def train_and_test(model_name, optimizer_name, device, epochs=10, lr=0.001):
    model = get_model(model_name).to(device)
    criterion = nn.CrossEntropyLoss()

    if optimizer_name == "SGD":
        optimizer = optim.SGD(model.parameters(), lr=lr)
    else:
        optimizer = optim.Adam(model.parameters(), lr=lr)

    start_time = time.time()
    model.train()
    for _ in range(epochs):
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

    train_time_ms = (time.time() - start_time) * 1000

    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

    test_acc = 100 * correct / total
    flops = compute_flops(model)

    return test_acc, train_time_ms, flops

In [1]:
models_list = ["resnet18", "resnet32", "resnet50"]
optimizers = ["SGD", "Adam"]
devices = [("CPU", device_cpu), ("GPU", device_gpu)]

results = []

for compute_name, device in devices:
    for optimizer in optimizers:
        for model_name in models_list:
            acc, time_ms, flops = train_and_test(
                model_name=model_name,
                optimizer_name=optimizer,
                device=device
            )

            results.append({
                "Compute": compute_name,
                "Batch Size": 16,
                "Optimizer": optimizer,
                "LR": 0.001,
                "Model": model_name,
                "Test Accuracy (%)": acc,
                "Train Time (ms)": time_ms,
                "FLOPs": flops
            })

            print(compute_name, optimizer, model_name, "Acc= ",acc, "Time=",time_ms,"FLOPS=", flops)


CPU SGD resnet18 Acc= 83.90 Time= 1103327 FLOPS= 1.751
CPU SGD resnet32 Acc= 20.56 Time= 4418103 FLOPS= 3.311
CPU SGD resnet50 Acc= 80.13 Time= 4065350 FLOPS= 3.967
CPU Adam resnet18 Acc= 75.27 Time= 1036824 FLOPS= 1.751
CPU Adam resnet32 Acc= 26.42 Time= 4508394 FLOPS= 3.311
CPU Adam resnet50 Acc= 79.46 Time= 3985297 FLOPS= 3.967
GPU SGD resnet18 Acc= 83.75 Time= 59704 FLOPS= 1.751
GPU SGD resnet32 Acc= 22.19 Time= 163156 FLOPS= 3.311
GPU SGD resnet50 Acc= 82.26 Time= 132047 FLOPS= 3.967
GPU Adam resnet18 Acc= 80.77 Time= 52771 FLOPS= 1.751
GPU Adam resnet32 Acc= 18.73 Time= 164316 FLOPS= 3.311
GPU Adam resnet50 Acc= 74.06 Time= 136962 FLOPS= 3.967


